###### working directory: G:\My Drive\Data\Voting Age Data Census 2018 5yr\Notebooks

# Analysis of voting age population for the 50th California Congressional District.

  


This jupyter notebook analyzes the voting age for the 50th California Congressional District. The geography of the district is mostly comprised of San Diego County but also includes several census tracts in Riverside County. This notebook produces bar charts to examine:  
* the top ten census tracts with the highest population for all ages
* the top ten census tracts with the highest concentration of voters over the age of 29, and;
* the top ten census tracts with the highest level of voters under the age of 30.

*Data source: U.S. Census Bureau, 2014-2018 American Community Survey 5-Year Estimates; Table S2902, Citizen, voting-age population by selected characteristics.* 

In [1]:
import numpy as np
import pandas as pd
import math

pd.options.display.float_format = '{:.2f}'.format
# pd.set_option('display.max_columns', 30) #can specify for rows too, and change number to maximum you want

In [2]:
pd.__version__

'1.0.3'

### Calling in the dataset


Calling in the voter dataset. Creating a reference dataframe to view the columns to decide which to keep and drop

In [3]:
# call in the dataset
ref = pd.read_csv('G:\My Drive\Data\Voting Age Data Census 2018 5yr\Original_Data\ACSST5Y2018.S2902_data_with_overlays_2020-03-03T174358.csv', delimiter = ',')

# avoid renaming columns, exploring column names.
# truncate the dataset, to two lines. column name and descriptions.
ref = ref.head()

In [4]:
ref = ref.head(2)
ref

,GEO_ID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M,...,S2902_C02_018E,S2902_C02_018M,S2902_C01_019E,S2902_C01_019M,S2902_C02_019E,S2902_C02_019M,S2902_C01_020E,S2902_C01_020M,S2902_C02_020E,S2902_C02_020M
0,id,Geographic Area Name,Estimate!!Total!!Citizens 18 years and over,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!A...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!P...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!P...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...
1,1400000US06065030101,"Census Tract 301.01, Riverside County, California",883,148,(X),(X),464,130,52.5,9.7,...,20.0,12.4,706,132,80.0,12.4,58710,6512,(X),(X)


In [5]:
ref.to_csv(r'G:\My Drive\Data\Voting Age Data Census 2018 5yr\Temp_Trash\Est_Header_.csv')

In [6]:
#specify datatypes so variables read in properly
variable_types = {'Geo.id': 'str', 'GEO.id2': 'str'}

#specify dtype with variable_types we created above
df = pd.read_csv('G:\My Drive\Data\Voting Age Data Census 2018 5yr\Original_Data\ACSST5Y2018.S2902_data_with_overlays_2020-03-03T174358.csv', delimiter = ',', 
                     header=[0], skiprows=[1], dtype=variable_types)

In [7]:
df.sort_values(by=['S2902_C01_001E'], ascending=False).head(1)

,GEO_ID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M,...,S2902_C02_018E,S2902_C02_018M,S2902_C01_019E,S2902_C01_019M,S2902_C02_019E,S2902_C02_019M,S2902_C01_020E,S2902_C01_020M,S2902_C02_020E,S2902_C02_020M
959,1400000US06073018700,"Census Tract 187, San Diego County, California",29371,1760,(X),(X),24413,1850,83.1,3.5,...,5.3,3.4,12060,1144,94.7,3.4,42126,5240,(X),(X)


In [8]:
# new data frame splitting GEO_ID to column 1. Nulls may create an error
new = df["GEO_ID"].str.split("S", n = 1, expand = True) 

# splitting GEO_ID column to use with lookup table later. 
df["GEOID"]= new[1] 

In [9]:
# produce a list of colums with quotes
cols = list(df.columns.values)

# Moving the newly created GEOID from the last position to the first postion. 
cols = cols[-1:] + cols[:-1]

# Committing the changes to the df dataframe.
df = df[cols]

In [10]:
df.head(1)

,GEOID,GEO_ID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,...,S2902_C02_018E,S2902_C02_018M,S2902_C01_019E,S2902_C01_019M,S2902_C02_019E,S2902_C02_019M,S2902_C01_020E,S2902_C01_020M,S2902_C02_020E,S2902_C02_020M
0,06065030101,1400000US06065030101,"Census Tract 301.01, Riverside County, California",883,148,(X),(X),464,130,52.5,...,20.0,12.4,706,132,80.0,12.4,58710,6512,(X),(X)


In [11]:
# dropping columns by label
df=df.drop(['GEO_ID'], axis = 1)

##### Calling in a lookup table for the census tracts which define the 50th congressional district. This will be merged with primary dataset and to drop census tracts outside the 50th congression district. 

In [12]:
#specify datatypes so GEOID readS in properly
variable_types = {'GEOID': 'str'}

# call in the dataset lookup table for the 50th district only
lookup = pd.read_csv('G:\My Drive\Data\CA Congress Dist 50\Data_Exports\CA_Congress_Lookup_50th.csv', delimiter = ',', dtype=variable_types)

# dropping columns by label
lookup=lookup.drop(['FID'], axis = 1)

In [13]:
lookup

,GEOID
0,06065051200
1,06065049600
2,06073020029
3,06073020309
4,06073021205
...,...
154,06073016606
155,06073019002
156,06073016503
157,06073016504


In [14]:
# Joining voter data to the look up table and dropping census tracts outside of the congressional district. 
dist_50 = pd.merge(df, lookup, on='GEOID', how='inner')
dist_50.head(1)

,GEOID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M,...,S2902_C02_018E,S2902_C02_018M,S2902_C01_019E,S2902_C01_019M,S2902_C02_019E,S2902_C02_019M,S2902_C01_020E,S2902_C01_020M,S2902_C02_020E,S2902_C02_020M
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,(X),(X),1343,366,31.0,6.9,...,10.8,5.6,3862,484,89.2,5.6,64321,13079,(X),(X)


In [15]:
ref

,GEO_ID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M,...,S2902_C02_018E,S2902_C02_018M,S2902_C01_019E,S2902_C01_019M,S2902_C02_019E,S2902_C02_019M,S2902_C01_020E,S2902_C01_020M,S2902_C02_020E,S2902_C02_020M
0,id,Geographic Area Name,Estimate!!Total!!Citizens 18 years and over,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!A...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!P...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!P...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...
1,1400000US06065030101,"Census Tract 301.01, Riverside County, California",883,148,(X),(X),464,130,52.5,9.7,...,20.0,12.4,706,132,80.0,12.4,58710,6512,(X),(X)


##### Dropping columns but keeping data for: 
* total population
* ages 18 to 29
* ages 29 and over

Dropping all other columns

In [16]:
# keep the first 10 columns and drop the rest. 
dist_50_2 = dist_50.drop(dist_50.iloc[:, 10:82:1], axis=1)

# Dropping columns S2902_C02_001E, S2902_C02_001M. 
dist_50_2 = dist_50_2.drop(dist_50.iloc[:, 4:6:1], axis=1)

dist_50_2.head(1)

,GEOID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,1343,366,31.0,6.9


##### Verifying the merge dropped census tracts outside the district. 

In [17]:
# number of rows and columns in df
total_rows=len(dist_50_2.axes[0])
total_cols=len(dist_50_2.axes[1])
print("Number of Rows: "+str(total_rows))
print("Number of Columns: "+str(total_cols))

Number of Rows: 159
Number of Columns: 8


In [18]:
# make list of column names
cols = list(dist_50_2.columns.values)
cols

['GEOID',
 'NAME',
 'S2902_C01_001E',
 'S2902_C01_001M',
 'S2902_C01_002E',
 'S2902_C01_002M',
 'S2902_C02_002E',
 'S2902_C02_002M']

##### Renaming columns:

In [19]:
# Renameing columns
dist_50_2.rename(columns={   'S2902_C01_001E':'tot_ovr_18',
                             'S2902_C01_001M': 'tot_ovr_18_moe',
                             'S2902_C01_002E': 'tot_18_29',
                             'S2902_C01_002M': 'tot_18_29_moe',
                             'S2902_C02_002E': 'pct_18_29',
                             'S2902_C02_002M': 'pct_18_29_moe'}, inplace=True)

##### Sorting data

In [20]:
dist_50_2.sort_values(by=['tot_ovr_18'], ascending=False)
dist_50_2.head(2)

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,1343,366,31.0,6.9
1,06065043218,"Census Tract 432.18, Riverside County, California",4002,338,959,221,24.0,4.4


In [21]:
# getting data types
dist_50_2.dtypes

GEOID             object
NAME              object
tot_ovr_18         int64
tot_ovr_18_moe     int64
tot_18_29          int64
tot_18_29_moe      int64
pct_18_29         object
pct_18_29_moe     object
dtype: object

#####   changing strings to numeric data types

In [22]:
# changing strings to numeric data types
dist_50_2["pct_18_29"] = pd.to_numeric(dist_50_2["pct_18_29"])
dist_50_2["pct_18_29_moe"] = pd.to_numeric(dist_50_2["pct_18_29_moe"])

#####  Calculating the number of those over 29 and under 30

In [23]:
# Calculating the number of those over 29
dist_50_2['tot_ovr_30'] = dist_50_2['tot_ovr_18'] - dist_50_2['tot_18_29']

# finding the inverse percentage of those over 29 years old.
dist_50_2['pct_ovr_30'] = 100 - dist_50_2['pct_18_29']

dist_50_2.head(4)

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe,tot_ovr_30,pct_ovr_30
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,1343,366,31.00,6.90,2988,69.00
1,06065043218,"Census Tract 432.18, Riverside County, California",4002,338,959,221,24.00,4.40,3043,76.00
2,06065043220,"Census Tract 432.20, Riverside County, California",2970,300,952,252,32.10,6.40,2018,67.90
3,06065043222,"Census Tract 432.22, Riverside County, California",3591,275,552,147,15.40,3.80,3039,84.60


In [24]:
# sorting data
dist_50_2.sort_values(by=['tot_ovr_18'], ascending=False).head(3)

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe,tot_ovr_30,pct_ovr_30
74,06073018700,"Census Tract 187, San Diego County, California",29371,1760,24413,1850,83.10,3.50,4958,16.90
103,06073020027,"Census Tract 200.27, San Diego County, California",10622,688,840,348,7.90,3.00,9782,92.10
93,06073020013,"Census Tract 200.13, San Diego County, California",9611,534,1480,386,15.40,3.80,8131,84.60


In [25]:
cols = list(dist_50_2.columns.values)
cols

['GEOID',
 'NAME',
 'tot_ovr_18',
 'tot_ovr_18_moe',
 'tot_18_29',
 'tot_18_29_moe',
 'pct_18_29',
 'pct_18_29_moe',
 'tot_ovr_30',
 'pct_ovr_30']

##### Creating a census tract label column for use in graphs

In [26]:
# new data frame splitting Census tract number to column 2. Nulls may create an error
new = dist_50_2["NAME"].str.split(" ", n = 5, expand = True) 
new = new[2].str.split(",", n = 5, expand = True) 
new.head(2)

,0,1
0,432.16,
1,432.18,


In [27]:
#Creating new column from split for CT. 
dist_50_2["Census Tract"]= new[0]
dist_50_2['Census Tract'] = 'CT ' + dist_50_2['Census Tract'].astype(str)
dist_50_2.head(2)

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe,tot_ovr_30,pct_ovr_30,Census Tract
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,1343,366,31.00,6.90,2988,69.00,CT 432.16
1,06065043218,"Census Tract 432.18, Riverside County, California",4002,338,959,221,24.00,4.40,3043,76.00,CT 432.18


In [28]:
# new data frame splitting County to column 1. Nulls may create an error
new = dist_50_2["NAME"].str.split(" ", n =3, expand = True) 
new = new[3].str.split(" C", n =1, expand = True)

# Creating new County column. 
dist_50_2["County"]= new[0] 

In [29]:
dist_50_2.head(1)

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe,tot_ovr_30,pct_ovr_30,Census Tract,County
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,1343,366,31.00,6.90,2988,69.00,CT 432.16,Riverside


##### Creating a lookup table. I need the census tract names and county for another project

In [30]:
# Selecting columns by name
dist_50_temp = dist_50_2[['GEOID', 'Census Tract', 'County']]

# export dataframe
dist_50_temp.to_csv('G:\My Drive\Data\Race Eth ACS 2017\Data_Exports\lookup_ct_county_name.csv', index=False)
dist_50_temp.head(2)

,GEOID,Census Tract,County
0,06065043216,CT 432.16,Riverside
1,06065043218,CT 432.18,Riverside


### Exporting data for:

*Voters over 18

*Voters over 18 to 29

*Voters over 30

In [31]:
dist_50_2.head(2)

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe,tot_ovr_30,pct_ovr_30,Census Tract,County
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,1343,366,31.00,6.90,2988,69.00,CT 432.16,Riverside
1,06065043218,"Census Tract 432.18, Riverside County, California",4002,338,959,221,24.00,4.40,3043,76.00,CT 432.18,Riverside


In [30]:
# exporting data
dist_50_2.to_csv(r'G:\My Drive\Data\Voting Age Data Census 2018 5yr\Data_Exports\eligible_voters_over_18.csv', index =False)